In [1]:
import os
import sys
import torch

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
 
if os.path.abspath(os.path.join(os.getcwd(), "../../")) not in sys.path:
    sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../../")))
    
from plan_generator.src.config import Configuration
from plan_generator.src.models import PlanGenerator
from plan_generator.src.data import PlanDataset
from plan_generator.src.trainer import PlanGeneratorTrainer

In [ ]:
# Train the base model to learn the data distribution

configuration = Configuration()

configuration.TRAIN_SIZE = 1.0
configuration.VALIDATION_SIZE = 0.0
configuration.TEST_SIZE = 0.0

plan_dataset = PlanDataset(configuration=configuration, use_transform=False)
plan_generator = PlanGenerator(configuration=configuration)

plan_generator_trainer = PlanGeneratorTrainer(
    configuration=configuration,
    plan_generator=plan_generator,
    plan_dataset=plan_dataset,
    existing_log_dir=os.path.abspath("plan_generator/runs/01-09-2025__15-49-24"),
    validating=False,
)

print(f"Devices: {[torch.cuda.get_device_name(i) for i in range(torch.cuda.device_count())]}")

plan_generator_trainer.fit()

In [ ]:
# Train the above model using augmented data

configuration = Configuration()

configuration.TRAIN_SIZE = 0.6
configuration.VALIDATION_SIZE = 0.4
configuration.TEST_SIZE = 0.0

plan_dataset = PlanDataset(configuration=configuration, end=70000, use_transform=True)
plan_generator = PlanGenerator(configuration=configuration)

plan_generator_trainer = PlanGeneratorTrainer(
    configuration=configuration,
    plan_generator=plan_generator,
    plan_dataset=plan_dataset,
    existing_log_dir=os.path.abspath("plan_generator/runs/01-09-2025__15-49-24"),
    validating=True,
)

print(f"Devices: {[torch.cuda.get_device_name(i) for i in range(torch.cuda.device_count())]}")

plan_generator_trainer.fit()